In [ ]:
!pip install -U 'farm-haystack[pinecone]'>=1.3.0 pinecone-client datasets

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
# Windows
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-windows-x86_64.zip -q
! Expand-Archive -Path <SourcePathofZipFile> -DestinationPath <DestinationPath>

In [16]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
# ! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
# ! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
# ! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
import time
es_server = Popen(
    ["./models/elasticsearch-7.9.2/bin/elasticsearch.bat"], stdout=PIPE, stderr=STDOUT,  start_new_session=True # as daemon
)
# wait until ES has started
time.sleep(30)





In [18]:
import os
import time

from haystack.document_stores import ElasticsearchDocumentStore

# Wait 30 seconds only to be sure Elasticsearch is ready before continuing
time.sleep(30)

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index='haystack-lfqa',
    similarity="cosine",
    embedding_dim=768
)

Traceback (most recent call last):
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 255, in perform_request
    response = self.pool.urlopen(
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\ur

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

In [17]:
from haystack.document_stores import ElasticsearchDocumentStore
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index='haystack-lfqa',
    similarity="cosine",
    embedding_dim=768
)

Traceback (most recent call last):
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 255, in perform_request
    response = self.pool.urlopen(
  File "C:\Users\Public\ACI_Proj\Projects\question-pair\venv\lib\site-packages\ur

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

In [ ]:
import re; 
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open("./data/aci_business.txt")
data = fp.read()
data = re.sub("\n+", "\n", data)
docs =[{"content": paragraph} for paragraph in data.splitlines()]

In [ ]:
# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(docs)

In [ ]:
import torch
# confirm GPU is available, outputs True if so
torch.cuda.is_available()

In [ ]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers"
)

In [ ]:
document_store.update_embeddings(
   retriever,
   batch_size=128
)

Updating embeddings:   0%|          | 0/286 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipe = DocumentSearchPipeline(retriever)
result = search_pipe.run(
    query="Is ACI  be a good company?",
    params={"Retriever": {"top_k": 2}}
)

print_documents(result)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Is ACI  be a good company?

{   'content': 'With almost three decades of partnering life and engendering '
               'hope, ACI is one of the top pharmaceutical companies in '
               'Bangladesh, employing more than 5,000 people all over the '
               'country. As a progressive and forward-thinking company, ACI '
               'Pharma is dedicated to improve the health of people of '
               'Bangladesh through introduction of innovative and reliable '
               'pharmaceutical products.',
    'name': None}

{   'content': 'Advanced Chemical Industries (ACI) Limited is one of the '
               'leading conglomerates in Bangladesh, with a multinational '
               'heritage. We have a mission to achieve business excellence '
               'through quality by understanding, accepting, meeting and '
               'exceeding customer expectations. We follow International '
               'Standards on Quality Management System to ensure co

In [ ]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

In [ ]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:


result = pipe.run(
        query="Mission of ACI?",
        params={
            "Retriever": {"top_k": 3},
            "Generator": {"top_k": 1}
        })

result["answers"][0].answer



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[<Answer {'answer': 'The mission of ACI is to improve the quality of life of the people through responsible application of knowledge, technology and skills.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['65076eca9c09331e0c268ff6939a3801', '85d8843deade9d2f617f90defa8184de', 'f86673619fc2091f1347d3732cd52380'], 'doc_scores': [0.501715093273254, 0.5015258702631026, 0.5013632466219664], 'content': ["ACI's Mission is to enrich the quality of life of the people through responsible application of knowledge, technology and skills. ACI is committed to the pursuit of excellence through world-class products, innovative processes and empowered employees, to provide the highest level of satisfaction to our customers.", 'The vision of ACI are Provide products and services of high and consistent quality, ensuring value for money to our customers, Endeavour to attain a position of leadership i

In [ ]:
result["answers"][0].answer

'The mission of ACI is to improve the quality of life of the people through responsible application of knowledge, technology and skills.'

In [ ]:
result.keys()

dict_keys(['query', 'answers', 'documents', 'root_node', 'params', 'node_id'])

In [ ]:
result["documents"]

[<Document: {'content': 'With almost three decades of partnering life and engendering hope, ACI is one of the top pharmaceutical companies in Bangladesh, employing more than 5,000 people all over the country. As a progressive and forward-thinking company, ACI Pharma is dedicated to improve the health of people of Bangladesh through introduction of innovative and reliable pharmaceutical products.', 'content_type': 'text', 'score': 0.5016781436987151, 'meta': {}, 'embedding': None, 'id': '88f16502772e9c62fb74182e0cd36423'}>,
 <Document: {'content': 'Advanced Chemical Industries (ACI) Limited is one of the leading conglomerates in Bangladesh, with a multinational heritage. We have a mission to achieve business excellence through quality by understanding, accepting, meeting and exceeding customer expectations. We follow International Standards on Quality Management System to ensure consistent quality of products and services to achieve customer satisfaction.', 'content_type': 'text', 'scor